In [8]:
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
from pandas import ExcelWriter

missing_values = ["n/a", "na", "--", " ","","NaN", "nan", None]
pd.set_option('display.max_columns', None, 'display.max_rows', None)
dfForEncodedAnswers = pd.read_excel("dfForEncodedAnswers.xls")
dfForTextAnswers = pd.read_excel("dfForTextAnswers.xls")
indicatedData = pd.read_excel("indicatedData.xls")

display(HTML("<style>.container { width:100% !important; }</style>"))

In [9]:
dfForEncodedAnswers.drop(['Unnamed: 0'], axis=1, inplace=True)
dfForTextAnswers.drop(['Unnamed: 0'], axis=1, inplace=True)

dimensions = ['Date','Region','Operateur','Service','Entreprise','Type Commercial','Question','Type Reponse','Reponse TXT','Reponse ENC']
answers = pd.DataFrame(columns=dimensions)
answers.shape

(0, 10)

In [10]:
typeRepMultiple = []
typeRepNormal = []
for col in dfForTextAnswers.columns:
    if(dfForTextAnswers[col].dtype == 'object'):
        typeRepNormal.append(col)
toRemove = []
toRemove.append('3 - [4] S0.1 Code ville')
toRemove.append('7 - [14] Date de passation du questionnaire')
toRemove.append('10 - [15] S1ter. Dans quel secteur d’activité travaillez-vous ?')
toRemove.append("15 - [20] S5. Et quel est l’opérateur principal, c'est-à-dire, l’opérateur mobile chez qui vous avez souscrit le plus de lignes mobiles ? ")
toRemove.append('40 - [49] Q05B. Par quels principaux moyens avez-vous trouvé les informations que vous recherchiez ?')
toRemove.append('42 - [52] Q05C. Par quels principaux moyens avez-vous reçu ces informations de la part de votre opérateur ?')
toRemove.append('47 - [59] Q07B. Pour cette démarche, avez-vous…')


columnsArray = []
for col in dfForTextAnswers.columns :
    columnsArray.append(col)

for toRm in toRemove:
    typeRepNormal.remove(toRm)
    columnsArray.remove(toRm)

typeRepMultiple.append('40 - [49] Q05B. Par quels principaux moyens avez-vous trouvé les informations que vous recherchiez ?')
typeRepMultiple.append('42 - [52] Q05C. Par quels principaux moyens avez-vous reçu ces informations de la part de votre opérateur ?')
columnsArray.append('40 - [49] Q05B. Par quels principaux moyens avez-vous trouvé les informations que vous recherchiez ?')
columnsArray.append('42 - [52] Q05C. Par quels principaux moyens avez-vous reçu ces informations de la part de votre opérateur ?')

columns = pd.DataFrame(columnsArray, columns=['question'])

writer = ExcelWriter('indicatedQuestions.xls')
columns.to_excel(writer,'columns')
writer.save()

In [11]:
entry=[]
question=''
repText=''
repEnc=''
commR1 = ''
commR2 = ''
typeComm =''
typeRep = ''
typeService =''
rowNum = 0

for i in range(1,len(dfForEncodedAnswers)):
    rowNum += 1
    commR1 = dfForEncodedAnswers.at[i,'48 - [61] Q08.Globalement, sur une échelle de 1 à 10, comment évaluez-vous la visite de ce commercial?']
    commR2 = dfForEncodedAnswers.at[i,'53 - [66] Q11.1 L’accueil privilégié en tant que client entreprise']
    if( (commR1 != 999) and (commR2 != 999) ):
        typeComm = 'Les Deux'
    elif( (commR1 != 999) and (commR2 == 999) ):
        typeComm = 'Agent Commercial'
    elif( (commR1 == 999) and (commR2 != 999) ):
        typeComm = 'Visite'
    else :
        typeComm = 'Aucun'
    
    date= dfForEncodedAnswers['7 - [14] Date de passation du questionnaire'][i]
    region = dfForEncodedAnswers['3 - [4] S0.1 Code ville'][i]
    oper= dfForEncodedAnswers['15 - [20] S5. Et quel est l’opérateur principal, c\'est-à-dire, l’opérateur mobile chez qui vous avez souscrit le plus de lignes mobiles ? '][i]
    secteur= dfForEncodedAnswers['10 - [15] S1ter. Dans quel secteur d’activité travaillez-vous ?'][i]
    
    
    for j in range(5,len(dfForEncodedAnswers.columns)):
        rep = dfForEncodedAnswers.columns[j]
        if(rep in typeRepNormal):
            typeRep = 'Normal'
        elif(rep in typeRepMultiple):
            typeRep = 'Choix Multiple'
        else:
            typeRep = 'Recommandation'
        
        if(j>=7 and len(indicatedData)):
            questionTmp = indicatedData.at[j-7, indicatedData.columns[1]]
            if ("roaming" in questionTmp):
                typeService = 'Service Roaming'
            elif("[service internet]" in questionTmp):
                typeService = 'Service Internet'
            elif("[Facturation]" in questionTmp):
                typeService = 'Service Facturation'
            elif("[service telephonique]" in questionTmp):
                typeService = 'Service Telephonique'
            elif("[paiement]" in questionTmp):
                typeService = 'Service Paiement'
            else :
                typeService = 'Autres'

                       
        if not pd.isna(dfForTextAnswers.at[i,dfForTextAnswers.columns[j]]):
                                   
            repText= dfForTextAnswers.at[i,dfForTextAnswers.columns[j]]
            repEnc = dfForEncodedAnswers.at[i, dfForEncodedAnswers.columns[j]]
            question=dfForEncodedAnswers.columns[j]
            entry= [date, region, oper, typeService, secteur,typeComm, question,typeRep, repText,repEnc]
            answers.loc[len(answers)]= entry 
            


In [12]:
answers.loc[answers['Service'] == '', ['Service'] ] = 'Autres'

In [13]:
for i in range(len(answers)):
    answers.at[i,'Reponse TXT'] = str(answers.at[i,'Reponse TXT'])

In [39]:
firstPartAnswers = answers.drop(range(0,35000))
secondPartAnswers = answers.drop(range(35001, len(answers)))

In [42]:
writer = ExcelWriter('FirstPartAnswers.xls')
firstPartAnswers.to_excel(writer,'firstPart')
writer.save()

writer = ExcelWriter('SecondPartAnswers.xls')
secondPartAnswers.to_excel(writer,'secondPart')
writer.save()

In [62]:
answers['Reponse TXT'].loc[answers['Reponse ENC'] == 107].unique()
answers['Question'].loc[answers['Reponse ENC'] == 107].unique()

array(['40 - [49] Q05B. Par quels principaux moyens avez-vous trouvé les informations que vous recherchiez ?',
       '42 - [52] Q05C. Par quels principaux moyens avez-vous reçu ces informations de la part de votre opérateur ?'],
      dtype=object)

In [57]:
answers.columns

Index(['Date', 'Region', 'Operateur', 'Service', 'Entreprise',
       'Type Commercial', 'Question', 'Type Reponse', 'Reponse TXT',
       'Reponse ENC'],
      dtype='object')